# Sql options with regular spark calls


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('SQL OPTIONS').getOrCreate()
spark

In [4]:
crime = spark.read.csv('rec-crime-pfa.csv',header = True,inferSchema=True)

In [5]:
crime.limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling year total number of offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [6]:
crime.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [7]:
df = crime.withColumnRenamed('Rolling year total number of offences','Rolling_year_total_number_of_offences')

In [8]:
df.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling_year_total_number_of_offences: integer (nullable = true)



In [9]:
df.createOrReplaceTempView('tempview')

In [10]:
spark.sql('SELECT * FROM tempview').limit(5).toPandas()


,12 months ending,PFA,Region,Offence,Rolling_year_total_number_of_offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [11]:
sql_results = spark.sql('SELECT * FROM tempview WHERE Rolling_year_total_number_of_offences > 1000').limit(5).toPandas()


In [12]:
spark.sql('SELECT Region,Offence FROM tempview WHERE Region LIKE "North%" ').limit(5).toPandas()

,Region,Offence
0,North West,All other theft offences
1,North West,Bicycle theft
2,North West,Criminal damage and arson
3,North West,Death or serious injury caused by illegal driving
4,North West,Domestic burglary


In [13]:
sql_results

,12 months ending,PFA,Region,Offence,Rolling_year_total_number_of_offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561
4,31/03/2003,Avon and Somerset,South West,Drug offences,2308


In [14]:
spark.sql('SELECT Region, sum(Rolling_year_total_number_of_offences) as Counts FROM tempview GROUP BY Region').limit(5).toPandas()

,Region,Counts
0,Fraud: CIFAS,7678981
1,North West,30235732
2,British Transport Police,3029117
3,Wales,11137260
4,London,42691902


In [15]:
from pyspark.ml.feature import SQLTransformer

In [16]:
sqltrans = SQLTransformer(statement="SELECT PFA,Region,Offence,Rolling_year_total_number_of_offences FROM __THIS__ WHERE Offence LIKE '%theft'")

In [17]:
sqltrans.transform(df).show()

+--------------------+--------------------+-------------+-------------------------------------+
|                 PFA|              Region|      Offence|Rolling_year_total_number_of_offences|
+--------------------+--------------------+-------------+-------------------------------------+
|   Avon and Somerset|          South West|Bicycle theft|                                 3090|
|        Bedfordshire|                East|Bicycle theft|                                  719|
|British Transport...|British Transport...|Bicycle theft|                                 2737|
|      Cambridgeshire|                East|Bicycle theft|                                 3975|
|            Cheshire|          North West|Bicycle theft|                                 1551|
|      City of London|              London|Bicycle theft|                                  427|
|           Cleveland|          North East|Bicycle theft|                                 1028|
|             Cumbria|          North We

In [18]:
from pyspark.sql.functions import expr

In [19]:
spark.sql('SELECT sum(Rolling_year_total_number_of_offences) FROM tempview').show()

+------------------------------------------+
|sum(Rolling_year_total_number_of_offences)|
+------------------------------------------+
|                                 244720928|
+------------------------------------------+



In [20]:
df.withColumn("percent",expr('round((Rolling_year_total_number_of_offences/244720928 * 100),3)')).show(10)

+----------------+-----------------+----------+--------------------+-------------------------------------+-------+
|12 months ending|              PFA|    Region|             Offence|Rolling_year_total_number_of_offences|percent|
+----------------+-----------------+----------+--------------------+-------------------------------------+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|                                25959|  0.011|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft|                                 3090|  0.001|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|                                26202|  0.011|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|                                    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|                                14561|  0.006|
|      31/03/2003|Avon and Somerset|South West|       Drug offences|            

In [21]:
df.select('*',expr('round((Rolling_year_total_number_of_offences/244720928 * 100),3) AS PCT_CHANGE')).limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling_year_total_number_of_offences,PCT_CHANGE
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.011
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.001
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.011
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.000
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.006


In [22]:
df.selectExpr('*','round((Rolling_year_total_number_of_offences/244720928 * 100),3) as PCT_CHANGE').limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling_year_total_number_of_offences,PCT_CHANGE
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.011
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.001
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.011
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.000
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.006


In [23]:
df.select('*',expr('round((Rolling_year_total_number_of_offences/244720928 * 100),3) as PCT_CHANGE')).filter('Region = "Wales" ').limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling_year_total_number_of_offences,PCT_CHANGE
0,31/03/2003,Dyfed-Powys,Wales,All other theft offences,2635,0.001
1,31/03/2003,Dyfed-Powys,Wales,Bicycle theft,230,0.000
2,31/03/2003,Dyfed-Powys,Wales,Criminal damage and arson,4890,0.002
3,31/03/2003,Dyfed-Powys,Wales,Death or serious injury caused by illegal driving,9,0.000
4,31/03/2003,Dyfed-Powys,Wales,Domestic burglary,666,0.000


# HW

In [24]:
googl_df = spark.read.csv('googleplaystore.csv',header = True , inferSchema=True)

In [25]:
googl_df.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [26]:
googl_df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [27]:
from pyspark.sql.types import IntegerType, FloatType

In [28]:
new_df = googl_df.withColumn('Rating',googl_df['Rating'].cast(FloatType())) \
        .withColumn('Reviews',googl_df['Reviews'].cast(IntegerType()))\
        .withColumn('Price',googl_df['Price'].cast(IntegerType()))

In [29]:
new_df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [30]:
new_df.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [31]:
new_df.createOrReplaceTempView('view')

In [32]:
new_df[new_df.Rating> 4.1].select('APP').limit(5).toPandas()

,APP
0,"U Launcher Lite – FREE Live Cool Themes, Hide ..."
1,Sketch - Draw & Paint
2,Pixel Draw - Number Art Coloring Book
3,Paper flowers instructions
4,Garden Coloring Book


In [33]:
app_rating = spark.sql('SELECT App FROM view WHERE Rating > 4.1').collect()

In [34]:
app_rating[:5]


[Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps'),
 Row(App='Sketch - Draw & Paint'),
 Row(App='Pixel Draw - Number Art Coloring Book'),
 Row(App='Paper flowers instructions'),
 Row(App='Garden Coloring Book')]

In [35]:
app_rating = new_df[new_df.Rating> 4.1].select('APP').collect()

In [36]:
app_rating[:5]

[Row(APP='U Launcher Lite – FREE Live Cool Themes, Hide Apps'),
 Row(APP='Sketch - Draw & Paint'),
 Row(APP='Pixel Draw - Number Art Coloring Book'),
 Row(APP='Paper flowers instructions'),
 Row(APP='Garden Coloring Book')]

In [37]:
spark.sql('SELECT App,Reviews FROM view ORDER BY Reviews desc  ').show(5)

+------------------+--------+
|               App| Reviews|
+------------------+--------+
|          Facebook|78158306|
|          Facebook|78128208|
|WhatsApp Messenger|69119316|
|WhatsApp Messenger|69119316|
|WhatsApp Messenger|69109672|
+------------------+--------+
only showing top 5 rows



In [38]:
spark.sql('SELECT App FROM view WHERE App LIKE "%dating%"').show(5,False)

+--------------------------------------------------+
|App                                               |
+--------------------------------------------------+
|Meet, chat & date. Free dating app - Chocolate app|
|Friend Find: free chat + flirt dating app         |
|Spine- The dating app                             |
|Princess Closet : Otome games free dating sim     |
|happn – Local dating app                          |
+--------------------------------------------------+



In [39]:
spark.sql('SELECT Genres,count(*) as Total FROM view GROUP BY Genres ORDER BY Total desc').show()

+-----------------+-----+
|           Genres|Total|
+-----------------+-----+
|            Tools|  842|
|    Entertainment|  623|
|        Education|  549|
|          Medical|  463|
|         Business|  460|
|     Productivity|  424|
|           Sports|  398|
|  Personalization|  392|
|    Communication|  387|
|        Lifestyle|  381|
|          Finance|  366|
|           Action|  365|
| Health & Fitness|  340|
|      Photography|  335|
|           Social|  295|
| News & Magazines|  283|
|         Shopping|  260|
|   Travel & Local|  257|
|           Dating|  234|
|Books & Reference|  231|
+-----------------+-----+
only showing top 20 rows



In [50]:
spark.sql('SELECT * FROM view').limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [57]:
new_df.groupBy('Genres').count().orderBy('Count',ascend = True).show(5,False)

+---------------------------------+-----+
|Genres                           |count|
+---------------------------------+-----+
|Books & Reference;Creativity     |1    |
|Tools;Education                  |1    |
|Arcade;Pretend Play              |1    |
|Parenting;Brain Games            |1    |
|Travel & Local;Action & Adventure|1    |
+---------------------------------+-----+
only showing top 5 rows

